# Laboratoire 5 - Odométrie

> **ATTENTION!** Ne pas faire rouler le robot sur les tables


> **ATTENTION!** Il est *strictement interdit* de s'approcher des commandes qui font avancer le robot si il y a le moindre risque que le robot ne tombe par terre. Les équipes qui ont un accidents de robot devront chanter *[cette chanson](https://www.youtube.com/watch?v=dQw4w9WgXcQ)* devant toute la classe.


> **PROTIP** En cas de problème, soulever le robot l'empêche d'avancer


> **ATTENTION!** Ne pas faire rouler le robot sur les tables

Au cours de ce laboratoire nous allons étudier la façon dont le robot se déplace. Comme vous le constaterez, le sujet est plus complexe qu'il n'y paraît! Connectons-nous d'abord au robot.


In [ ]:
%matplotlib inline

import time
from time import sleep

import math
import matplotlib.pyplot as plt
import numpy as np
from math import pi as PI

from robmob.robot import Robot
from robmob.rover.sensors import RobotEspSensor
from robmob.rover.commands import MovementFloatCommand, ResetCommand

robot_ip = 'localhost'
robot = Robot(robot_ip)
robot.connect()

## Partie 1 - Le problème du contrôle

Pour cette première expérience, utilisez les fonctions de la librairie `robmob` pour ordonner au robot d'avancer d'un mètre.

In [ ]:
robot.linear_movement(0.2, 5)  # Ordonne au robot d'avancer à 0.2 m/s pendant 5 s

Vous constaterez que les rovers se déplacent de moins qu'un mètre avec l'exécution de cette commande. Qu'est-ce qui se passe? Pour le comprendre, il faut d'abord se demander comment le robot exécute notre demande.

Nous envoyons une commande de vitesse au robot. Le robot active ses moteurs, puis après un certain $\Delta t$ le robot consulte les encodeurs de ses roues pour valider sa vitesse. La différence entre la vitesse demandée et la vitesse réelle constitue une erreur. Cette erreur est fournie à une fonction appelée un [correcteur PID](https://fr.wikipedia.org/wiki/R%C3%A9gulateur_PID) qui traduit l'erreur en une nouvelle consigne. Cette nouvelle consigne tente de corriger l'erreur sur la vitesse du robot, pour exécuter aussi bien que possible la commande qu'on lui a demandé.

Malheureusement, il est difficile de créer un correcteur parfait. Dans le cas qui nous concerne, le correcteur semble avoir de la difficulté à atteindre la vitesse demandée, surtout au début de la trajectoire. Voici un graphique de la vitesse instantanée d'un robot rover par rapport au temps. La vitesse demandée est en rouge. L'apparence de ce graphique explique pourquoi les distances parcourues par le robot sont plus courtes que les distances demandées.

![Vitesse de la kobuki par rapport au temps](img/vitesse_kobuki.png)



## Partie 2 - Rotation

Dans cette section nous effectuerons des rotations et nous validerons comment le robot peut déduire son état à l'aide de ses différents senseurs.


En premier lieu, fixez une règle sur le robot et placez une feuille de papier de façon à pouvoir facilement calculer des angles, comme sur l'image.

![Montage rotation](img/montage_rotation.png)

Maintenant demandons au robot de faire une rotation de $\pi$ radians. 

In [ ]:
robot.angular_movement(PI / 2, 4)  # Demande au robot de tourner à PI/2 radians/seconde pendant 4 secondes.

Mesurez la rotation que le robot a effectué. Vous constaterez que la rotation résultante est beaucoup plus faible que la rotation demandée.  Mais le robot en est-il conscient?

### Odométrie

Nous allons utiliser l'odomètre du robot pour estimer sa rotation réelle. Pour ce faire nous pouvons ajouter un `OdometerTicksSensor` à notre objet `robot`. L'odomètre nous renvoie trois valeurs: le *timestamp* du message, le nombre de ticks enregistrés pour la roue gauche, et le nombre de ticks enregistrés par la roue droite.

In [ ]:
sensor = RobotEspSensor()
robot.add_sensor(sensor)
sleep(1)
sensor.peek_odom()

<mark>Valider le format du data (ticks, meter, etc.)</mark>
Il est possible de convertir ces ticks en distance parcourue! En sachant que l'encodeur produit 2048 ticks par tour de roue, il est possible de trouver le nombre de ticks par mètre de déplacement. Avec cette valeur, il est possible de convertir des ticks en mètre!

In [ ]:
message = sensor.peek_odom()
TICK_TO_METER = ... # TODO Trouver le facteur de conversion entre ticks et mètres
displacement = ... # TODO calculer le déplacement en mètre pour la roue gauche
print(displacement)

Le code suivant demande au robot de faire une rotation de $\pi$ radians, et enregistre le status de l'odomètre avant et après le déplacement. Sachant cela, calculez le déplacement subi par la roue gauche et la roue droite.

In [ ]:
odometer_before = sensor.peek_odom()
robot.angular_movement(PI / 2, 2)
sleep(1)
odometer_after = sensor.peek_odom()

In [ ]:
displacement_right = ...  # TODO calculer le deplacement de la roue droite en metres
displacement_left = ...  # TODO calculer le deplacement de la roue droite en metres

print('Left (m): {}, Right (m): {}'.format(displacement_left, displacement_right))

**Exercice**

À l'aide de ces déplacements, calculez la rotation que le robot a effectué, du point de vue de l'odomètre. La valeur `robot.DISTANCE_CENTER_TO_WHEEL` pourrait vous être utile.

In [ ]:
rotation = 0.0  # TODO

print('Rotation (rad): {}, Rotation (deg): {}'.format(rotation, rotation / (2 * PI) * 360))

### Gyroscope

Vous êtes déjà à l'aise avec le gyroscope. Nous allons intégrer ses mesures pour faire un nouvel estimé de la rotation. Mais d'abord, trouvons rapidement un biais. Pour ce faire, laissez le robot immobile pendant que vous exécutez le code suivant.

In [ ]:
gyro_data = sensor.sample_gyro_for_x_sec(20)
gyro_data = np.array([x[2] for x in gyro_data])  # On a seulement besoin de la vitesse angulaire autour de l'axe z.
bias = np.mean(gyro_data)

print('Bias (deg/s): {}'.format(bias))

Avec un peu de chance, le routeur va bien se comporter et vous recevez les mesures du gyro en temps réel. Si c'est le cas, exécutez du code de façon à intégrer le déplacement angulaire du robot alors qu'il exécute la même commande.

In [ ]:
sensor.buffer.clear()
sleep(2)
robot.angular_movement(PI / 2, 4)
sleep(1)

gyro_data = sensor.read_gyro()  # Le buffer contient seulement les X dernières secondes.
gyro_data = [x['z'] for x in gyro_data]

rotation = 0.0
for reading in gyro_data:
    rotation += (reading - bias) * (1. / sensor.SAMPLE_RATE)

print('Rotation (rad): {}, Rotation (deg): {}'.format(np.deg2rad(rotation), rotation))

## Partie 3 - Variance sur le déplacement

Codez une boucle qui demande au robot de faire une grande quantité (autour de 20) de petites rotations. À chaque rotation, calculez le déplacement angulaire du point de vue de l'odomètre. Finalement, calculez l'écart type de la distribution ainsi calculée. La fonction `np.var` calcule la variance d'une liste.

In [ ]:
def angle_from_odom_values(before, after):
    displacement_left = (after[1] - before[1]) * TICKS_TO_METER
    displacement_right = (after[2] - before[2]) * TICKS_TO_METER

    return (displacement_right - displacement_left) / (2 * robot.DISTANCE_CENTER_TO_WHEEL)


odom_measurements = []

for _ in range(20):
    # TODO - replacez le pass par votre code
    pass
    #1. Prendre une mesure initiale

    #2. Faire une commande et attendre l'exécutier

    #3. Prendre une nouvelle mesure

    #4. Calculer l'angle et l'ajouter à la liste odom_measurements


In [ ]:
odom_measurements = np.array(odom_measurements)
print('Measurements: ' + str(odom_measurements))
print('Standard deviation (rad): {}'.format(np.sqrt(np.var(odom_measurements))))

## Intermède - Les listes et numpy

Pour bien comprendre la partie 4, il faut comprendre que `numpy` (de façon similaire à matlab) permet de faire facilement des opérations sur des listes. Prenez bien le temps de comprendre les sorties suivantes avant de continuer.

In [ ]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([5, 4, 3, 2, 1])
c = np.array([[5, 4], [3, 2], [1, 2]])

print('a + b        =', a + b)
print('a * b        =', a * b)
print('a + 3        =', a + 3)
print('a * 2        =', a * 2)
print('a[2:4]       =', a[2:4])
print('c[:, 0]      =', c[:, 0])
print('c[:, 1]      =', c[:, 1])
print('np.diff(a)   =', np.diff(a))
print('np.cumsum(a) =', np.cumsum(a))

## Partie 4 - Intégration

Vous avez maintenant tous les outils qu'il vous faut pour estimer la position du robot dans un repère global à travers le temps. Pour ce faire il suffit d'intégrer la vitesse du robot à travers le temps. 

Commençons par collecter des données. Débutez l'exécution du code suivant. Il s'assure que le buffer de l'odométrie a une durée d'une minute. Ensuite, utilisez le fichier [Controller.ipynb](Controller.ipynb) pour conduire le robot sur une trajectoire de votre choix. Déplacez le robot pendant un peu moins d'une minute, de sorte que toutes les données nécessaires se trouveront dans le buffer. Après `duration_of_data_collection` secondes les données du buffer seront capturées et vous pourrez les analyser.

> N'hésitez pas à utiliser la commande `plt.plot()` sur n'importe quelle liste dans votre code pour aider le debugging.

In [ ]:
duration_of_data_collection = 30

robot = Robot(robot_ip)
robot.connect()

sensor = RobotEspSensor(int(RobotEspSensor.SAMPLE_RATE * duration_of_data_collection))

robot.add_sensor(sensor)

sleep(duration_of_data_collection)

odometry_data = sensor.read_odom()

À partir de `odometry_data`, on doit créer quatre listes. Une liste doit contenir le temps écoulé depuis le début de la collecte de données. Une liste doit contenir le temps écoulé entre les mesures successives d'odométrie. Une autre liste doit contenir le nombre de ticks d'odomètre qui ont eu lieu pendant ce temps pour la roue gauche. Le dernier vecteur doit contenir la même chose, mais pour la roue droite. La fonction `np.diff()` vous sera très utile ici.

In [ ]:
TICK_TO_METER = np.pi * (80. / 1000.) / 2048.
t = odometry_data[:, 0] - odometry_data[0, 0]
delta_t = np.diff(t) + 1e-6  # To avoid division by zero
displacement_left = np.diff(odometry_data[:, 1]) * TICK_TO_METER
displacement_right = np.diff(odometry_data[:, 2]) * TICK_TO_METER

On peut déduire la vitesse de chaque roue à travers le temps de la façon suivante.

In [ ]:
right_wheel_speed = displacement_right / delta_t
left_wheel_speed = displacement_left / delta_t

En utilisant les équations de conduite différentielle vues en classe, on peut convertir les vitesses des roues en des vitesses linéaires et angulaires du robot, pour chaque temps $t$.

In [ ]:
linear_speed = (right_wheel_speed + left_wheel_speed) / 2
angular_speed = (right_wheel_speed - left_wheel_speed) / (2 * robot.DISTANCE_CENTER_TO_WHEEL)

Finalement, convertissons ces vitesses en une position $\vec{x}(t)$ et une orientation $\vec{\theta}(t)$. Vous devrez *intégrer* les vitesses pour y arriver. 

> **PROTIP** Il faut d'abord intégrez l'orientation! Vous en avez besoin pour calculer la position x y...

In [ ]:
theta_of_t = np.cumsum(angular_speed * delta_t)
x_of_t = np.cumsum(linear_speed * np.cos(theta_of_t) * delta_t)
y_of_t = np.cumsum(linear_speed * np.sin(theta_of_t) * delta_t)

Affichez la trajectoire calculée avec les commandes suivante.

> **NOTE** La trajectoire affichée commence en foncé et fini en pâle.

In [ ]:
n_of_colors = math.floor(x_of_t.shape[0] / 100)

colors = np.linspace(0, 1, n_of_colors) ** 2

for color, xs, ys in zip(colors, np.array_split(x_of_t, n_of_colors), np.array_split(y_of_t, n_of_colors)):
    plt.plot(xs, ys, color=(0.2, color, color))

plt.axis('equal')  # On s'assure que l'échelle est la même sur les deux axes

plt.show()

## Partie 5 - Modèle idéal de conduite différentielle VS *Skid Steer*

Le rover ne suit pas le modèle idéal de conduite différentielle puisque les roues doivent déraper afin d'effectuer la rotation. Pour en arriver à un modèle plus représentatif du rover, il faut ajouter un coefficient de glissent aux équations utilisées pour intégrer les données d'odométrie. L'intégration de l'odométrie a été pour inclure ce coefficient afin d'améliorer votre estimation du déplacement.

In [ ]:
duration_of_data_collection = 20

robot = Robot(robot_ip)
robot.connect()

sensor = RobotEspSensor(int(RobotEspSensor.SAMPLE_RATE * duration_of_data_collection))

robot.add_sensor(sensor)

# TODO Déplacer le robot avec l'aide de robot.general_mouvement afin d'effectuer une trajectoire en cercle.
# TODO Mesurer le diamètre de la trajectoire qui a été effectuée.

odometry_data = sensor.read_odom()

In [ ]:
# TODO Afficher la trajectoire comme à la partie précédente.
alpha = ...

TICK_TO_METER = np.pi * (80. / 1000.) / 2048.
t = odometry_data[:, 0] - odometry_data[0, 0]
delta_t = np.diff(t) + 1e-6  # To avoid division by zero
displacement_left = np.diff(odometry_data[:, 1]) * TICK_TO_METER
displacement_right = np.diff(odometry_data[:, 2]) * TICK_TO_METER

right_wheel_speed = displacement_right / delta_t
left_wheel_speed = displacement_left / delta_t

linear_speed = (right_wheel_speed + left_wheel_speed) / 2
angular_speed = alpha * (right_wheel_speed - left_wheel_speed) / (2 * robot.DISTANCE_CENTER_TO_WHEEL)

theta_of_t = np.cumsum(angular_speed * delta_t)
x_of_t = np.cumsum(linear_speed * np.cos(theta_of_t) * delta_t)
y_of_t = np.cumsum(linear_speed * np.sin(theta_of_t) * delta_t)

n_of_colors = math.floor(x_of_t.shape[0] / 100)

colors = np.linspace(0, 1, n_of_colors) ** 2

for color, xs, ys in zip(colors, np.array_split(x_of_t, n_of_colors), np.array_split(y_of_t, n_of_colors)):
    plt.plot(xs, ys, color=(0.2, color, color))

plt.axis('equal')  # On s'assure que l'échelle est la même sur les deux axes

plt.show()

Comparer la trajectoire à partir des données d'odométrie avec la trajectoire réelle. Est-ce que le diamètre est le même? 

Modifiez `alpha` pour que la trajectoire calculée corresponde à la trajectoire réelle. Quelle est la valeur de `alpha` qui vous permet d'obtenir la meilleure correspondance?
